In [8]:
import numpy as np
import xarray as xr
import geopy.distance
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy
import seaborn as snt
from datetime import datetime, timedelta
import pandas as pd
import cftime

In [9]:
lonMin, lonMax = -60, 10
latMin, latMax = 0, 31

In [10]:
#Open the file with the AEW tracks from August-September 2024
#ds = xr.open_dataset('/Users/awing/Dropbox/ORCESTRA/code/AEWtracking/AEW_tracks_BOWTIE_post_processed.nc',decode_times=False)
AEW = xr.open_dataset('../AEWtracking/AEW_tracks_BOWTIE_post_processed.nc')

AEW_lon2024 = AEW.AEW_lon_smooth  # System x Time
AEW_lat2024 = AEW.AEW_lat_smooth  # System x Time
AEWtime = AEW.time  #
#time2024 = ds['time'].values  # Time in seconds since 1970-01-01 00:00:00.0
#start_date2024 = cftime.DatetimeGregorian(1970, 1, 1, 0, 0, 0)  # Use cftime for high capacity date handling
AEW

<xarray.Dataset> Size: 543kB
Dimensions:         (latitude: 61, longitude: 151, time: 244, system: 25)
Coordinates:
  * latitude        (latitude) float64 488B 49.62 48.62 47.62 ... -9.375 -10.0
  * longitude       (longitude) float64 1kB -89.62 -88.62 -87.62 ... 59.38 60.0
  * time            (time) datetime64[ns] 2kB 2024-08-01 ... 2024-09-30T18:00:00
  * system          (system) float64 200B 1.0 2.0 3.0 4.0 ... 23.0 24.0 25.0
Data variables:
    AEW_lon         (system, time) float64 49kB ...
    AEW_lat         (system, time) float64 49kB ...
    AEW_lon_smooth  (system, time) float64 49kB ...
    AEW_lat_smooth  (system, time) float64 49kB ...
    AEW_strength    (system, time) float64 49kB ...
    TC_gen_time     (system) datetime64[ns] 200B ...
    TC_name         (system) <U3 300B ...
    curv_data_mean  (time, longitude) float64 295kB ...

In [11]:
cv = xr.open_dataset('../AEWtracking/curv_vort_era5_BOWTIE.nc')
cv

<xarray.Dataset> Size: 18MB
Dimensions:    (time: 244, latitude: 61, longitude: 151)
Coordinates:
  * time       (time) datetime64[ns] 2kB 2024-08-01 ... 2024-09-30T18:00:00
  * latitude   (latitude) float64 488B 49.62 48.62 47.62 ... -8.375 -9.375 -10.0
  * longitude  (longitude) float64 1kB -89.62 -88.62 -87.62 ... 58.38 59.38 60.0
Data variables:
    curv_vort  (time, latitude, longitude) float64 18MB ...

In [12]:
DSHIP = xr.open_dataset('../../data/meteor_meteo_dship_20240923.nc')
DSHIP

<xarray.Dataset> Size: 20MB
Dimensions:           (time: 58980)
Coordinates:
  * time              (time) datetime64[ns] 472kB 2024-08-14 ... 2024-09-23T2...
    lat               (time) float64 472kB ...
    lon               (time) float64 472kB ...
Data variables: (12/39)
    sst_port          (time) float64 472kB ...
    sst_extern_port   (time) float64 472kB ...
    sst_intern_port   (time) float64 472kB ...
    sst_board         (time) float64 472kB ...
    sst_extern_board  (time) float64 472kB ...
    sst_intern_board  (time) float64 472kB ...
    ...                ...
    ship_heave        (time) float64 472kB ...
    ship_heave_std    (time) float64 472kB ...
    ship_pitch        (time) float64 472kB ...
    ship_pitch_std    (time) float64 472kB ...
    ship_roll         (time) float64 472kB ...
    ship_roll_std     (time) float64 472kB ...

In [13]:
#time range
start_time = np.datetime64('2024-08-16T06:00:00')
index_start = np.where(pd.to_datetime(cv.time) == start_time)[0]
index_start = int(index_start[0])
iship_start = np.where(pd.to_datetime(DSHIP.time) == start_time)[0]
iship_start = int(iship_start[0])

end_time = np.datetime64('2024-09-24T00:00:00')
index_end = np.where(pd.to_datetime(cv.time) == end_time)[0]
index_end = int(index_end[0])


In [14]:
plt.figure(figsize = (18, 6))

count = 1
for iplot in range(index_start, index_end):

    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines(resolution = '50m',alpha=0.5)
    gl = ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1,alpha = 0.25)
    gl.top_labels = False
    gl.right_labels = False
    gl.xlocator = mticker.FixedLocator([-50, -40, -30, -20, -10, 0, 10])
    gl.ylocator = mticker.FixedLocator([0, 5, 10, 15, 20, 25, 30])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size'
                       : 18}
    gl.ylabel_style = {'size': 18}

    plt.contourf(cv.longitude.values-360.0, 
               cv.latitude.values,
               cv.curv_vort.values[iplot,:,:],
               cmap = 'RdGy_r', levels = np.linspace(-2e-5,2e-5,21), extend = 'both')

    cbar=plt.colorbar(orientation='vertical', pad=0.08)
    cbar.set_label('Curvature Vort. ($10^{-6} s^{-1}$)',fontsize=18)  # Optional: add a label to the colorbar

    plt.scatter(AEW_lon2024[:,iplot], AEW_lat2024[:,iplot], marker = '*', color = 'b', s = 200)
    iship = np.where(pd.to_datetime(DSHIP.time) == pd.to_datetime(cv.time)[iplot])[0]
    iship = int(iship[0])
    plt.scatter(DSHIP.lon[iship], DSHIP.lat[iship], marker = 'o', color = 'k', s = 100)
    plt.plot(DSHIP.lon[iship_start:iship+1], DSHIP.lat[iship_start:iship+1], 'k', linewidth = 1)

    #Find storms at time iplot and plot their tracks up to this point
    istorm = ~np.isnan(AEW_lat2024[:,iplot])
    for i in range(len(istorm)):
        if istorm[i]:
            plt.plot(AEW_lon2024[i,:iplot+1], AEW_lat2024[i,:iplot+1], 'b', linewidth = 1)


    cbar.ax.tick_params(labelsize=18)
    ax.set_xlim(lonMin, lonMax)
    ax.set_ylim(latMin, latMax)
    plt.title(pd.to_datetime(cv.time)[iplot], fontsize = 18)

    # Save the figure with the index in the file name
    plt.savefig(f'../figures/frames/fig_AEW_cv_{count:03d}.png',bbox_inches='tight')
    count = count+1

    # Clear the current axes to prepare for the next plot
    plt.clf()


<Figure size 1800x600 with 0 Axes>